In [1]:
import numpy as np
from scipy.ndimage import convolve

def convolve_gene_expression(data, kernel_size):
    """
    对给定的三维基因表达数据进行累加卷积操作。

    参数:
    - data: 一个形状为 (height, width, depth) 的 ndarray，代表基因表达数据。
    - kernel_size: 卷积核的大小，n 表示 n*n 的卷积核。

    返回:
    - 卷积后的基因表达数据，形状为 (height, width, depth) 的 ndarray。
    """
    # 创建一个 n*n 的卷积核，这里使用全1，代表简单的相加操作
    kernel = np.ones((kernel_size, kernel_size))

    # 初始化输出数据
    convolved_data = np.zeros(data.shape)

    # 对每个 depth (每个基因表达分量) 单独进行卷积
    for i in range(data.shape[-1]):
        # 注意：convolve 函数自动处理边界，使用 mode='constant'，并且常数值为0
        convolved_data[:, :, i] = convolve(data[:, :, i], kernel, mode='constant', cval=0.0)

    return convolved_data

# 示例：
data = np.load("dataset/expression_matrix.npy",allow_pickle=True) # 假设数据
kernel_size = 5  # 比如 3*3 的卷积核
convolved_data = convolve_gene_expression(data, kernel_size)

In [ ]:
from sklearn.decomposition import PCA

def apply_pca_and_reshape(data, n_components):
    """
    对给定的基因表达数据应用PCA降维，并将结果重塑回原始的图像空间尺寸。

    参数:
    - data: 一个形状为 (height, width, depth) 的 ndarray，代表基因表达数据。
    - n_components: 降维后的维数。

    返回:
    - 降维并重塑后的基因表达数据，形状为 (height, width, n_components)。
    """
    # 重新塑形数据以适应PCA
    reshaped_data = data.reshape(-1, data.shape[-1])
    
    # 初始化PCA对象
    pca = PCA(n_components=n_components)
    
    # 对数据应用PCA
    pca_data = pca.fit_transform(reshaped_data)
    
    # 将降维后的数据重新塑形回原始图像的空间尺寸
    reshaped_pca_data = pca_data.reshape(data.shape[0], data.shape[1], n_components)

    return reshaped_pca_data

# 示例：
# 假设 convolved_data 是经过卷积操作的数据，形状为 (1200, 1200, 2000)
n_components = 50  # 假设你想将数据降维到50维
pca_and_reshaped_data = apply_pca_and_reshape(convolved_data, n_components)

np.save("dataset/expression_matrix.npy",pca_and_reshaped_data)